In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

2024-04-03 00:51:18.295235: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 00:51:18.321141: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 00:51:18.699947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
class EndToEndModel(tf.keras.Model):
    def __init__(self):
        super().__init__()

        # Convolutional layers
        self.conv_layers = tf.keras.Sequential([
            # First convolutional layer, Width: 200, Height: 66, Channels: 3
            layers.InputLayer(shape=(66, 200, 3)),
            layers.Conv2D(24, kernel_size=5, strides=2, padding='valid'),
            layers.BatchNormalization(),
            layers.ReLU(),

            # Second convolutional layer
            layers.Conv2D(36, kernel_size=5, strides=2, padding='valid'),
            layers.BatchNormalization(),
            layers.ReLU(),

            # Third convolutional layer
            layers.Conv2D(48, kernel_size=5, strides=2, padding='valid'),
            layers.BatchNormalization(),
            layers.ReLU(),

            # Fourth convolutional layer
            layers.Conv2D(64, kernel_size=3, strides=1, padding='valid'),
            layers.BatchNormalization(),
            layers.ReLU(),

            # Fifth convolutional layer
            layers.Conv2D(64, kernel_size=3, strides=1, padding='valid'),
            layers.BatchNormalization(),
            layers.ReLU(),
        ])

        # Flat and fully connected layers
        self.flat_layers = tf.keras.Sequential([
            # Flatten
            layers.Flatten(),
            layers.Dropout(0.5),

            # First fully connected layer
            layers.Dense(1164),
            layers.BatchNormalization(),
            layers.ReLU(),

            # Second fully connected layer
            layers.Dense(100),
            layers.BatchNormalization(),
            layers.ReLU(),

            # Third fully connected layer
            layers.Dense(50),
            layers.BatchNormalization(),
            layers.ReLU(),

            # Fourth fully connected layer
            layers.Dense(10),
            layers.ReLU(),  # Assuming a ReLU here for consistency with the PyTorch model

            # Output layer
            layers.Dense(1)
        ])

    def call(self, inputs):
        x = self.conv_layers(inputs)
        y = self.flat_layers(x)
        return y


In [4]:
def _parse_function(example_proto):
    # Define the feature description for parsing
    feature_description = {
        'steering': tf.io.FixedLenFeature([], tf.float32),
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    # Parse the input tf.train.Example proto using the dictionary above
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode the image
    image = tf.image.decode_jpeg(parsed_features['image'], channels=3)

    # Resize the image
    image_resized = tf.image.resize(image, [66, 200])  # Height x Width
    
    # Convert the image to YUV color space
    image_yuv = tf.image.rgb_to_yuv(image_resized)
    
    # Normalize the image data to [0, 1] (optional)
    image_normalized = image_yuv / 255.0

    return image_normalized, parsed_features['steering']


In [5]:
# Assuming you have already defined the EndToEndModel class

# Step 2: Create a TFRecord Dataset
file_paths = ["datasets/dataset_2024-04-03T06-00-42.tfrecord"]  # Update this to your TFRecord file path(s)
dataset = tf.data.TFRecordDataset(file_paths)

# Step 3: Map the Parsing Function to the Dataset
dataset = dataset.map(_parse_function)

# Step 4: Prepare the Dataset for Training
dataset = (
    dataset
    .shuffle(1000)
    .batch(32, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)


2024-04-03 00:51:21.701060: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 00:51:21.728306: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 00:51:21.728425: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
# Initialize your model
model = EndToEndModel()

# Compile the model (make sure to specify the loss and optimizer)
model.compile(optimizer='adam', loss='mean_squared_error')



In [ ]:
# Step 5: Train the Model
model.fit(dataset, epochs=10)

Epoch 1/10


I0000 00:00:1712130685.926110   72505 service.cc:145] XLA service 0x77453c01b6b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1712130685.926178   72505 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-04-03 00:51:25.972030: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


In [1]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

NameError: name 'tf' is not defined